In [23]:
import pandas as pd

# Load the dataset (you can adjust the file path)
file_path = 'Computer_Assisted_Mass_Appraisal_-_Residential.csv'
dataset = pd.read_csv(file_path)

# Drop columns with high percentage of missing values (like 'YR_RMDL')
dataset_cleaned = dataset.drop(columns=['YR_RMDL'])
dataset_cleaned = dataset_cleaned.dropna(subset=['PRICE'])
data = dataset_cleaned[dataset_cleaned['AC'] != 0]

# Encode AC Y/N to 1 and 0
dataset_cleaned['AC'] = dataset_cleaned['AC'].map({'Y': True, 'N': False})

# For numerical columns, fill missing values with the median (e.g., for 'PRICE', 'ROOMS', etc.)
# dataset_cleaned['PRICE'].fillna(dataset_cleaned['PRICE'].median(), inplace=True)
dataset_cleaned['ROOMS'].fillna(dataset_cleaned['ROOMS'].median(), inplace=True)
dataset_cleaned['BEDRM'].fillna(dataset_cleaned['BEDRM'].median(), inplace=True)
dataset_cleaned['KITCHENS'].fillna(dataset_cleaned['KITCHENS'].median(), inplace=True)
dataset_cleaned['FIREPLACES'].fillna(dataset_cleaned['FIREPLACES'].median(), inplace=True)

# For categorical columns, fill missing values with the mode (e.g., for 'STYLE', 'GRADE', etc.)
dataset_cleaned['STYLE'].fillna(dataset_cleaned['STYLE'].mode()[0], inplace=True)
dataset_cleaned['GRADE'].fillna(dataset_cleaned['GRADE'].mode()[0], inplace=True)
dataset_cleaned['STRUCT'].fillna(dataset_cleaned['STRUCT'].mode()[0], inplace=True)

# You can continue this process for other columns as needed

# Save the cleaned dataset if needed
# dataset_cleaned.to_csv('cleaned_dataset.csv', index=False)

# Check if there are any remaining missing values
print(dataset_cleaned.isnull().sum())
data = dataset_cleaned

SSL                     0
BATHRM               1806
HF_BATHRM            1806
HEAT                 1806
HEAT_D               1806
AC                   1841
NUM_UNITS            1806
ROOMS                   0
BEDRM                   0
AYB                    24
EYB                     0
STORIES              1974
SALEDATE                0
PRICE                   0
QUALIFIED               0
SALE_NUM                0
GBA                     0
BLDG_NUM                0
STYLE                   0
STYLE_D              1806
STRUCT                  0
STRUCT_D             1806
GRADE                   0
GRADE_D              1806
CNDTN                1806
CNDTN_D              1806
EXTWALL              1806
EXTWALL_D            1806
ROOF                 1806
ROOF_D               1806
INTWALL              1806
INTWALL_D            1806
KITCHENS                0
FIREPLACES              0
USECODE                 0
LANDAREA                0
GIS_LAST_MOD_DTTM       0
OBJECTID                0
dtype: int64

/var/folders/k2/hq4yd_p90y1blncf07bq7dx00000gn/T/ipykernel_93553/1177241130.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset_cleaned['ROOMS'].fillna(dataset_cleaned['ROOMS'].median(), inplace=True)
/var/folders/k2/hq4yd_p90y1blncf07bq7dx00000gn/T/ipykernel_93553/1177241130.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object o

### Drop unnessary column

In [24]:
data.drop(columns=['SSL', 'BLDG_NUM', 'SALE_NUM', 'HEAT_D', 'STYLE_D', 'STRUCT_D', 'GRADE_D', 'CNDTN_D', 'EXTWALL_D', 'ROOF_D', 'INTWALL_D', 'GIS_LAST_MOD_DTTM', 'OBJECTID'], inplace=True)
data.head()

,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,EYB,STORIES,...,STRUCT,GRADE,CNDTN,EXTWALL,ROOF,INTWALL,KITCHENS,FIREPLACES,USECODE,LANDAREA
0,4.0,1.0,8.0,True,1.0,12.0,6.0,1911.0,1989,3.75,...,6.0,8.0,4.0,20.0,11.0,6.0,1.0,6.0,11,2104
1,3.0,1.0,1.0,True,2.0,13.0,5.0,1912.0,1978,3.00,...,6.0,6.0,4.0,14.0,2.0,6.0,2.0,3.0,24,936
2,3.0,1.0,7.0,True,2.0,6.0,4.0,1910.0,1993,3.00,...,7.0,6.0,4.0,14.0,6.0,6.0,2.0,2.0,24,936
3,3.0,1.0,7.0,True,2.0,11.0,4.0,1912.0,1978,3.00,...,6.0,6.0,4.0,14.0,6.0,6.0,2.0,2.0,24,988
4,4.0,1.0,1.0,True,3.0,11.0,5.0,1912.0,1993,3.00,...,7.0,6.0,5.0,14.0,2.0,6.0,3.0,4.0,24,1674


### Modify date column

In [25]:
data['SALEDATE'] = pd.to_datetime(data['SALEDATE'])

# Remove the time part, keeping only the date (this will keep Year, Month, Day)
data['SALEDATE'] = data['SALEDATE'].dt.date

# Optionally, you can extract specific components from the date for model training
data['SALE_YEAR'] = pd.DatetimeIndex(data['SALEDATE']).year
data['SALE_MONTH'] = pd.DatetimeIndex(data['SALEDATE']).month

# Drop the original 'SALEDATE' column if not needed
data = data.drop(columns=['SALEDATE'])

In [27]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 96827 entries, 0 to 109033
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BATHRM       95021 non-null  float64
 1   HF_BATHRM    95021 non-null  float64
 2   HEAT         95021 non-null  float64
 3   NUM_UNITS    95021 non-null  float64
 4   ROOMS        96827 non-null  float64
 5   BEDRM        96827 non-null  float64
 6   AYB          96803 non-null  float64
 7   EYB          96827 non-null  int64  
 8   STORIES      94853 non-null  float64
 9   PRICE        96827 non-null  float64
 10  GBA          96827 non-null  int64  
 11  STYLE        96827 non-null  float64
 12  STRUCT       96827 non-null  float64
 13  GRADE        96827 non-null  float64
 14  CNDTN        95021 non-null  float64
 15  EXTWALL      95021 non-null  float64
 16  ROOF         95021 non-null  float64
 17  INTWALL      95021 non-null  float64
 18  KITCHENS     96827 non-null  float64
 19  FIREPLAC

,BATHRM,HF_BATHRM,HEAT,NUM_UNITS,ROOMS,BEDRM,AYB,EYB,STORIES,PRICE,...,KITCHENS,FIREPLACES,USECODE,LANDAREA,SALE_YEAR,SALE_MONTH,AC_False,AC_True,QUALIFIED_Q,QUALIFIED_U
0,4.0,1.0,8.0,1.0,12.0,6.0,1911.0,1989,3.75,3275000.0,...,1.0,6.0,11,2104,2019,8,False,True,True,False
1,3.0,1.0,1.0,2.0,13.0,5.0,1912.0,1978,3.00,550000.0,...,2.0,3.0,24,936,1999,8,False,True,False,True
2,3.0,1.0,7.0,2.0,6.0,4.0,1910.0,1993,3.00,1700000.0,...,2.0,2.0,24,936,2019,7,False,True,True,False
3,3.0,1.0,7.0,2.0,11.0,4.0,1912.0,1978,3.00,1500000.0,...,2.0,2.0,24,988,2021,10,False,True,True,False
4,4.0,1.0,1.0,3.0,11.0,5.0,1912.0,1993,3.00,2232500.0,...,3.0,4.0,24,1674,2023,4,False,True,True,False


In [29]:
data.columns

Index(['BATHRM', 'HF_BATHRM', 'HEAT', 'NUM_UNITS', 'ROOMS', 'BEDRM', 'AYB',
       'EYB', 'STORIES', 'PRICE', 'GBA', 'STYLE', 'STRUCT', 'GRADE', 'CNDTN',
       'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES', 'USECODE',
       'LANDAREA', 'SALE_YEAR', 'SALE_MONTH', 'AC_False', 'AC_True',
       'QUALIFIED_Q', 'QUALIFIED_U'],
      dtype='object')